In [1]:
import pandas as pd
import re
import csv

### Outlier decetion of internal data

In [2]:
# Converte json document to csv.
pdObj = pd.read_json('../data/curated/domain.json', orient='index')
pdObj.to_csv('../data/curated/streaming_test.csv', index=False)

In [3]:
# Get the cost
df = pd.read_csv('../data/curated/streaming_test.csv')
df['cost'] = df['cost_text']
for i in range(len(df)):
    df['cost'].iloc[i] = re.findall(r'\d+\.?\d*',df['cost_text'].iloc[i])

/Users/victor/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
# Delete missing values if cost is 0
delete_list = []
df_new = df.copy()
for i in range(len(df)):
    if len(df['cost'].iloc[i]) == 0:
        delete_list.append(i)
df_new = df_new.drop(df_new.index[delete_list])

In [5]:
# Transform the data type of cost into float.
for i in range(len(df_new)):
    df_new['cost'].iloc[i] = float(df_new['cost'].iloc[i][0])
    
df_new['cost'] = df_new['cost'].astype('float')

In [6]:
# Delete the extreme values: delete Highest 20% and lowest 20% cost
delete_list = []
k1 = 0.8
k2 = 0.2
df_new2 = df_new.copy()
for i in range(len(df_new2)):
    if df_new2['cost'].iloc[i] < df_new2['cost'].mean()*(1-k1) or df_new2['cost'].iloc[i] > df_new2['cost'].mean()*(1+k2):
        delete_list.append(i)
df_new2 = df_new2.drop(df_new2.index[delete_list])
df_new2.describe()

,postcode,cost
count,13231.000000,13231.000000
mean,3203.883531,474.439385
std,251.114949,137.840632
min,3000.000000,160.000000
25%,3044.000000,380.000000
50%,3125.000000,450.000000
75%,3204.000000,550.000000
max,3996.000000,931.000000


In [7]:
domain = df_new2

In [8]:
domain.head()

,name,cost_text,type,school_title,school_distance,suburb_sold,suburb_avg_day,suburb_population,suburb_age,suburb_ower,suburb_renter,suburb_family,suburb_single,coordinates,rooms,desc,postcode,cost
0,1202/620 Collins Street Melbourne VIC 3000,$500 per week,Apartment / Unit / Flat,University High School,2.4 km away,596,147,"47,279",20 to 39,30%,70%,24%,76%,"[-37.8186626, 144.9543471]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Ender Gok,3000,500.0
1,1701/620 Collins Street Melbourne VIC 3000,$650 per week,Apartment / Unit / Flat,University High School,2.4 km away,596,147,"47,279",20 to 39,30%,70%,24%,76%,"[-37.8186626, 144.9543471]","[['2 Beds'], ['2 Baths'], ['1 Parking']]",Ender Gok,3000,650.0
2,5707B/639 Little Lonsdale Street Melbourne VIC...,$570,Apartment / Unit / Flat,University High School,1.9 km away,387,120,"47,279",20 to 39,30%,70%,24%,76%,"[-37.8141942, 144.9521662]","[['1 Bed'], ['1 Bath'], []]",Kim Te,3000,570.0
3,1003/225 Elizabeth Street Melbourne VIC 3000,$575.00 per week,Apartment / Unit / Flat,Carlton Gardens Primary School,1.4 km away,596,147,"47,279",20 to 39,30%,70%,24%,76%,"[-37.8132308, 144.9623173]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Barbara Baltas,3000,575.0
4,1202/31 Spring Street Melbourne VIC 3000,$570.00,Apartment / Unit / Flat,Carlton Gardens Primary School,1.4 km away,596,147,"47,279",20 to 39,30%,70%,24%,76%,"[-37.8145047, 144.9740695]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Bo Natoli,3000,570.0


Some suburb data is missing in domian, this is probably because there are not enough property transactions in this postcide suburb on domian to be counted, so we try to fill in the data with neighbouring suburb, or delete them if there are no near suburb or only a few data is missing

In [9]:
domain[domain['suburb_population'].isnull()]['postcode'].value_counts()

3046    91
3161    86
3338    58
3004    57
3060    43
3024    39
3195    35
3335    30
3155    23
3337    22
3064    22
3012    21
3977    19
3550    15
3016    14
3940    13
3037    11
3043     9
3803     9
3026     8
3030     8
3021     7
3551     7
3690     5
3556     4
3220     2
3260     2
3121     2
3350     2
3756     1
3805     1
3915     1
3950     1
3971     1
3185     1
3215     1
3207     1
3196     1
3181     1
3135     1
3128     1
3084     1
3072     1
3041     1
3022     1
3015     1
3995     1
Name: postcode, dtype: int64

In [10]:
# 3000 data is filled in 3004
domain.loc[domain['postcode'] == 3004, 'suburb_sold'] = 596
domain.loc[domain['postcode'] == 3004, 'suburb_avg_day'] = 147
domain.loc[domain['postcode'] == 3004, 'suburb_population'] = 6423
domain.loc[domain['postcode'] == 3004, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3004, 'suburb_owner'] = '30%'
domain.loc[domain['postcode'] == 3004, 'suburb_renter'] = '70%'
domain.loc[domain['postcode'] == 3004, 'suburb_family'] = '24%'
domain.loc[domain['postcode'] == 3004, 'suburb_single'] = '76%'

In [11]:
domain.loc[domain['postcode'] == 3046, 'suburb_sold'] = 178
domain.loc[domain['postcode'] == 3046, 'suburb_avg_day'] = 48
domain.loc[domain['postcode'] == 3046, 'suburb_population'] = 22246
domain.loc[domain['postcode'] == 3046, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3046, 'suburb_owner'] = '63%'
domain.loc[domain['postcode'] == 3046, 'suburb_renter'] = '37%'
domain.loc[domain['postcode'] == 3046, 'suburb_family'] = '49%'
domain.loc[domain['postcode'] == 3046, 'suburb_single'] = '51%'

In [12]:
domain.loc[domain['postcode'] == 3024, 'suburb_sold'] = 288
domain.loc[domain['postcode'] == 3024, 'suburb_avg_day'] = 40
domain.loc[domain['postcode'] == 3024, 'suburb_population'] = 23294
domain.loc[domain['postcode'] == 3024, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3024, 'suburb_ower'] = '72%'
domain.loc[domain['postcode'] == 3024, 'suburb_renter'] = '28%'
domain.loc[domain['postcode'] == 3024, 'suburb_family'] = '51%'
domain.loc[domain['postcode'] == 3024, 'suburb_single'] = '49%'

In [13]:
domain.loc[domain['postcode'] == 3337, 'suburb_sold'] = 99
domain.loc[domain['postcode'] == 3337, 'suburb_avg_day'] = 43
domain.loc[domain['postcode'] == 3337, 'suburb_population'] = 17589
domain.loc[domain['postcode'] == 3337, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3337, 'suburb_ower'] = '74%'
domain.loc[domain['postcode'] == 3337, 'suburb_renter'] = '26%'
domain.loc[domain['postcode'] == 3337, 'suburb_family'] = '46%'
domain.loc[domain['postcode'] == 3337, 'suburb_single'] = '54%'

In [14]:
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_sold'].isnull()), 'suburb_sold'] = 97
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_avg_day'].isnull()),'suburb_avg_day'] = 14
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_population'].isnull()),'suburb_population'] = 8343
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_age'].isnull()),'suburb_age'] = '60+'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_ower'].isnull()), 'suburb_ower'] = '80%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_renter'].isnull()),'suburb_renter'] = '20%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_family'].isnull()),'suburb_family'] = '51%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_single'].isnull()),'suburb_single'] = '49%'

In [16]:
for post in domain.postcode:
    domain =  domain.drop(domain[(domain['postcode'] == post)&(domain['suburb_population'].isnull())].index)

In [ ]:
domain.to_csv('../data/curated/domain_final.csv',index=False)

### Locality to SA2 Coding Index
m

In [18]:
sa2_2016 = pd.read_csv("../data/raw/australian_postcodes.csv") # SA2 code in 2016
sa2_2021 = pd.read_csv("../data/raw/CG_SA2_2016_SA2_2021.csv")# SA2 code in 2021

In [ ]:
# merge data and only keep SA2 in VIC
index = sa2_2021.merge(sa2_2016, on='SA2_MAINCODE_2016')
index = index[index['state'] == 'VIC']

In [ ]:
# keep useful col and drop duplicates
index = index[['SA2_MAINCODE_2016', 'SA2_NAME_2016_x', 'SA2_CODE_2021', 'SA2_NAME_2021','postcode','locality','Lat_precise','Long_precise']]
index = index.drop_duplicates(subset=['postcode'])

In [ ]:
postcode = pd.read_csv('../data/raw/Australian-Postcode-Data-master/au_postcodes.csv') # postcode in VIC
code_vic = postcode[postcode['state_code'] == 'VIC']
code_vic = code_vic[postcode['accuracy'] == 4] # only keep accuracy is 4 postcode that refer to a suburb
code_vic = code_vic.drop_duplicates(subset=['postcode'])

In [ ]:
index_vic = code_vic[['postcode','latitude','longitude']].merge(index2, on='postcode')

In [ ]:
index_vic
index_vic.to_csv('../data/curated/postcode_to_sa2.csv',index=False)